In [1]:
import seaborn as sb
import matplotlib.pyplot as plt

sb.set()
sb.set_style("whitegrid")
sb.set_context("paper")
import pandas as pd
from os.path import join
from sklearn.manifold import TSNE
from scipy.linalg import norm
from oodtk.utils import is_known


root = "/home/ki/projects/work/mchad/logs/"
path = join(root, "runs/2021-12-10/15-18-02/")
# /home/ki/projects/work/mchad/logs/multiruns/complete/missing/12/09/21-16:26:33/cifar10/gmchad/0/tb/version_0/15680/test-1/
# home/ki/projects/work/mchad//tb/version_0/
for c in range(1):
    for i in [7]:
        print(c, i)
        p = join(path, f"tb/version_0/15680/test-{i}/")
        df = pd.read_csv(join(p, "tensors.tsv"), delimiter="\t", header=None)
        df_meta = pd.read_csv(join(p, "metadata.tsv"), delimiter="\t", header=0)
        df_meta["label"] = df_meta["labels"]

        df_meta["known"] = is_known(df_meta["labels"])
        df_meta["correct"] = df_meta["labels"] == df_meta["predictions"]

        df_meta["Group"] = 0
        df_meta["Group"][df_meta["known"] & df_meta["correct"]] = "Inlier (Correct Class)"
        df_meta["Group"][df_meta["known"] & ~df_meta["correct"]] = "Inlier (Wrong Class)"
        df_meta["Group"][~df_meta["known"]] = "Anomaly"

        print(df_meta)
        pca = TSNE()
        x = pca.fit_transform(df.values)

        sb.set_palette(sb.color_palette("tab10"))

        fix = plt.figure(figsize=(6, 6))
        for label in df_meta["label"].unique():
            if label < 0:
                continue
            idx = df_meta["label"] == label
            plt.scatter(x[idx, 0], x[idx, 1], label=label, s=1)

        idx = df_meta["label"] < 0
        plt.scatter(x[idx, 0], x[idx, 1], label=-1, s=1, color="black")
        sb.despine(left=True, bottom=True)
        plt.show()
        # plt.savefig("out/latent-cifar-lsun-resize.pgf")
        # plt.legend()

        norms = norm(df.values, ord=2, axis=1)
        label = pd.Series(
            data=(df_meta["label"] >= 0).apply({True: "Normal", False: "Anomaly"}.get),
            name="Label",
        )

        sb.kdeplot(x=norms, hue=df_meta["Group"], common_norm=False, fill=True)
        plt.xlabel("$\Vert f(x) \Vert $")
        sb.despine()
        plt.savefig("out/density_norm.pgf")
        plt.show()
        # df_meta["dist_e"] = np.log(df_meta["distance"])

        ax = sb.kdeplot(
            x=df_meta["distance"], hue=df_meta["Group"], common_norm=False, fill=True, bw=0.1
        )
        plt.xlabel("$\min_y \Vert f(x) - \mu_y \Vert$")
        # ax.legend_._set_loc(2)
        # plt.xlim([0.0, 1.3])
        # plt.xlim([0, 2])
        sb.despine()

        # plt.legend(loc="upper right")
        plt.savefig("out/density_distance.pgf")
        plt.show()

0 7


FileNotFoundError: [Errno 2] No such file or directory: '/home/ki/projects/work/mchad/logs/runs/2021-12-10/15-18-02/tb/version_0/15680/test-7/tensors.tsv'